# Dataset

## Import Library

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# visualization
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")
import os
import glob

# pytorch
import torch
from torch import optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
import torch.nn as nn

# sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

# others
import numpy as np
import pandas as pd
import pdb
import sys
sys.path.append("../")
from all_models import seed_everything,DNN
import evaluation_and_visualization as ev_viz
from all_models import MTNNConv_PressureVelocity

# Load Data

In [6]:
TRAINPERCENTAGE=0.55
DATASETNAME="../../../ParticleDragForceEstimation/datasets/evaluation_data/AllData_with_Pressure_And_Velocity_Samples_case123/AllData_PRESSURE_and_VELOCITY_10_SAMPLES_All_RE_SF_Groups_TRAINPERCENTAGE_{}_With_Output_Mask_".format(TRAINPERCENTAGE)
exists = os.path.isfile(DATASETNAME+"trainX.npy")
if exists:
    X_train=np.load(DATASETNAME+"trainX.npy")
    y_train=np.load(DATASETNAME+"trainY.npy")
    X_val=np.load(DATASETNAME+"valX.npy")
    y_val=np.load(DATASETNAME+"valY.npy")
    X_test=np.load(DATASETNAME+"testX.npy")
    y_test=np.load(DATASETNAME+"testY.npy")
    print("Loaded Files")
else:
    pass 

Loaded Files


In [7]:
# y matrix structure
#(Fx-nondim, PressureField1,...,PressureField10,VelocityXField1,...,VelocityXField10,Px,Py,Pz,TauX,TauY,Tauz,AvgFx-NonDim-Per_Re_SF,Mask)
#Currently Mask is all ones.
print("Shape of Training and Testing Files")
print(X_train.shape,y_train.shape,X_val.shape,y_val.shape,X_test.shape,y_test.shape)

Shape of Training and Testing Files
(2639, 47) (2639, 29) (564, 47) (564, 29) (2621, 47) (2621, 29)


## Normalization

In [8]:
#Mask specifically for pressure and velocity fields.
mask_train=np.int32(y_train[:,-1])
mask_val=np.int32(y_val[:,-1])
mask_test=np.int32(y_test[:,-1])

# normalize on training set and apply to test set
std_scaler_x = StandardScaler()
std_scaler_y = StandardScaler()
std_scaler_pres=StandardScaler()
std_scaler_vel=StandardScaler()
std_scaler_presdragcomponents=StandardScaler()
std_scaler_veldragcomponents=StandardScaler()

# fit and transform on training set
X_train = std_scaler_x.fit_transform(X_train)
y_train[:, 0] = std_scaler_y.fit_transform(
    np.reshape(y_train[:, 0], (y_train.shape[0], 1))
)[:, 0]
  
#apply transformation on validation set.
X_val = std_scaler_x.transform(X_val)
y_val[:,0] = std_scaler_y.transform(
     np.reshape(y_val[:,0], (y_val.shape[0],1))
   )[:,0]

# apply transformation on test set
X_test = std_scaler_x.transform(X_test)
y_test[:, 0] = std_scaler_y.transform(
    np.reshape(y_test[:, 0], (y_test.shape[0], 1))
)[:, 0]

# # # #Standard Scaling of pressure y values. Only calculated for training.

y_train[mask_train==1,1:11] = std_scaler_pres.fit_transform(y_train[mask_train==1,1:11])
y_val[mask_val==1,1:11] = std_scaler_pres.transform(y_val[mask_val==1,1:11])
y_test[mask_test==1,1:11] = std_scaler_pres.transform(y_test[mask_test==1,1:11])


#### Standard Scaling of velocity y values.Only calculated for training.

y_train[mask_train==1,11:21] = std_scaler_vel.fit_transform(y_train[mask_train==1,11:21])
y_val[mask_val==1,11:21] = std_scaler_vel.transform(y_val[mask_val==1,11:21]) 
y_test[mask_test==1,11:21] = std_scaler_vel.transform(y_test[mask_test==1,11:21])

##### Standard Scaling Of Pressure Drag Components.

y_train[:,21:24] = std_scaler_presdragcomponents.fit_transform(y_train[:,21:24])
y_val[:,21:24] = std_scaler_presdragcomponents.transform(y_val[:,21:24])
y_test[:,21:24] = std_scaler_presdragcomponents.transform(y_test[:,21:24])


###### Standard Scaling of Velocity Drag Components.

y_train[:,24:27] = std_scaler_veldragcomponents.fit_transform(y_train[:,24:27])
y_val[:,24:27] = std_scaler_veldragcomponents.transform(y_val[:,24:27])
y_test[:,24:27] = std_scaler_veldragcomponents.transform(y_test[:,24:27])

print("Shapes y_train = {}, y_val = {}, y_test = {}".format(y_train.shape,y_val.shape,y_test.shape))

Shapes y_train = (2639, 29), y_val = (564, 29), y_test = (2621, 29)


# Create A Separate Y_pdfmeans Numpy Array To Explicitly Constrain Distribution Means Of Pressure and Velocity Values

In [9]:
re_train=X_train[:,-2]
sf_train=X_train[:,-1]
tmppres=y_train[:,1:11]
tmpvel=y_train[:,11:21]
_tmp=np.hstack([re_train[:,None],sf_train[:,None],tmppres,tmpvel])
columns=["Re","Solidfraction"]+["Pressure_X_{}".format(i+1) for i in range(10)]+["Velocity_X_{}".format(i+1) for i in range(10)]
_tmpdftrain=pd.DataFrame(_tmp,columns=columns)


prescols=["Pressure_X_{}".format(i+1) for i in range(10)]
velcols=["Velocity_X_{}".format(i+1) for i in range(10)]
pdf_means=dict()
pdf_meansvel=dict()
for key,val in _tmpdftrain.groupby(['Re','Solidfraction']):
    pdf_means[key]=val[prescols].mean().mean()
    pdf_meansvel[key]=val[velcols].mean().mean()
    
y_pdfmean=list()
for row in _tmpdftrain.iterrows():
    _re=row[1][0]
    _sf=row[1][1]
    y_pdfmean.append([pdf_means[(_re,_sf)],pdf_meansvel[(_re,_sf)]])
y_pdfmean = np.array(y_pdfmean)
print(y_pdfmean.shape)

(2639, 2)


# Fully Connected Network 

## Configurations

In [14]:
# CUDA support 
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# model settings
D_in  = X_train.shape[1]
output_size = 1    #1 FX-nondim value.
output_size_aux1 = 10  #10 Pressure Sampled values.
output_size_aux2 = 10  #30 Velocity sampled values.
output_size_pdrag = 3
output_size_vdrag = 3

H = 128
depth = 1
depth_aux = 1
shared_depth = 1
NUMEPOCHS = 500
Batch_size = 100

## Model

## Training

In [ ]:
models=[]
#Best Parameters = 'presParam': 0.0001, 'velParam': 0.001, 'PDrag': 0.01, 'VDrag': 0.01, 'parampdfpresmean': 1.0, 'parampdfvelmean': 1.0
# for parampres in [0.01,0.001,0.0001]:
#     for paramvel in [0.1,0.01,0.001]:
for parampres in [0.0001]:
    for paramvel in [0.001]:
        for paramdrag in [0.01]:
            test_mses=list()
            test_mres=list()
            test_preds_all=list()
            test_targets_all=list()
            best_model=None
            val_epochs=100
            seed_everything(123)


            param1=parampres
            param2=paramvel
            parampdrag=paramdrag
            paramvdrag=paramdrag
            parampdfpresmean=1.0
            parampdfvelmean=1.0
            
            hyperparameters={'presParam':param1,'velParam':param2,'PDrag':parampdrag,'VDrag':paramvdrag,
                      'parampdfpresmean':parampdfpresmean,'parampdfvelmean':parampdfvelmean}

            
            losses=list()
            # Compile model   
            model = MTNNConv_PressureVelocity(D_in, H, output_size,
                        output_size_aux1,output_size_aux2,output_size_pdrag,output_size_vdrag, depth,depth_aux,
                                              shared_depth,device=device,numpresdragcomponent=3,numsheardragcomponent=3).to(device)

            # Loss Function
            criterion = torch.nn.MSELoss() 

            # Optimizer
            optimizer = optim.Adadelta(model.parameters())

            # Input Data
            trainX = Variable(torch.from_numpy(X_train).float()).to(device)
            trainY = Variable(torch.from_numpy(y_train[:,0]).float()).to(device)
            trainMask=torch.FloatTensor(torch.from_numpy(y_train[:,-1]).float()).to(device)
            trainY_AUX1 = Variable(torch.from_numpy(y_train[:,1:11]).float()).to(device)
            trainY_AUX2 = Variable(torch.from_numpy(y_train[:,11:21]).float()).to(device)
            trainY_PDrag = Variable(torch.from_numpy(y_train[:,21:24]).float()).to(device)
            trainY_VDrag = Variable(torch.from_numpy(y_train[:,24:27]).float()).to(device)
            trainY_PDFPRESMEANS = Variable(torch.from_numpy(y_pdfmean[:,0]).float()).to(device)
            trainY_PDFVELMEANS = Variable(torch.from_numpy(y_pdfmean[:,1]).float()).to(device) 

            valX = Variable(torch.from_numpy(X_val).float()).to(device)
            valY = Variable(torch.from_numpy(y_val[:,0]).float()).to(device)
            valMask=torch.FloatTensor(torch.from_numpy(y_val[:,-1]).float()).to(device)
            valY_AUX1 = Variable(torch.from_numpy(y_val[:,1:11]).float()).to(device)
            valY_AUX2 = Variable(torch.from_numpy(y_val[:,11:21]).float()).to(device)
            valY_PDrag = Variable(torch.from_numpy(y_val[:,21:24]).float()).to(device)
            valY_VDrag = Variable(torch.from_numpy(y_val[:,24:27]).float()).to(device)

            testX = Variable(torch.from_numpy(X_test).float()).to(device)
            testY = Variable(torch.from_numpy(y_test[:,0]).float()).to(device)
            testMask=torch.FloatTensor(torch.from_numpy(y_test[:,-1]).float()).to(device)
            testY_AUX1 = Variable(torch.from_numpy(y_test[:,1:11]).float()).to(device)
            testY_AUX2 = Variable(torch.from_numpy(y_test[:,11:21]).float()).to(device)
            testY_PDrag = Variable(torch.from_numpy(y_test[:,21:24]).float()).to(device)
            testY_VDrag = Variable(torch.from_numpy(y_test[:,24:27]).float()).to(device)

            # Train the model
            data_train_loader = DataLoader(
                list(zip(trainX,trainY,trainY_AUX1,trainY_AUX2,trainY_PDrag,trainY_VDrag,trainMask,trainY_PDFPRESMEANS,
                         trainY_PDFVELMEANS)), 
                batch_size=Batch_size, 
                shuffle=True)
            
            data_val_loader = DataLoader(list(zip(valX,valY,valY_AUX1,valY_AUX2,valY_PDrag,valY_VDrag,valMask)),
                                batch_size=Batch_size,shuffle=True)
            
            #Start Training
            for epoch in range(NUMEPOCHS):
                alltargets = list()
                allpredictions = list()
                allpredictions_aux_pres=list()
                allpredictions_aux_vel=list()
                allpredictions_pdrag=list()
                allpredictions_vdrag=list()

                epoch_losses_all=list()
                epoch_losses_main=list()
                epoch_losses_aux1=list()
                epoch_losses_aux2=list()
                epoch_losses_pdrag=list()
                epoch_losses_vdrag=list()

                for batchX, batchY,batchY_AUX_PRES,batchY_AUX_VEL,batchY_PDrag,batchY_VDrag,batchMask,batchY_PDFPRESMEAN,batchY_PDFVELMEAN in data_train_loader:

                    # Forward pass
                    outputs,outputs_AUX_PRES,outputs_AUX_VEL,outputs_Pdrag,outputs_Vdrag = model(batchX)

                    outputs_AUX_PRES=outputs_AUX_PRES*batchMask.unsqueeze(1)
                    outputs_AUX_VEL=outputs_AUX_VEL*batchMask.unsqueeze(1)
                    
                    loss1 = criterion(outputs.squeeze(), batchY)
                    pres_loss = criterion(outputs_AUX_PRES,batchY_AUX_PRES)
                    vel_loss = criterion(outputs_AUX_VEL,batchY_AUX_VEL)
                    pdrag_loss = criterion(outputs_Pdrag,batchY_PDrag)
                    vdrag_loss = criterion(outputs_Vdrag,batchY_VDrag)
                    pdfpresmean_loss = criterion(outputs_AUX_PRES.mean(dim=1),batchY_PDFPRESMEAN.squeeze())
                    pdfvelmean_loss = criterion(outputs_AUX_VEL.mean(dim=1),batchY_PDFVELMEAN.squeeze())

                    loss = loss1 + param1*pres_loss + param2*vel_loss + parampdrag*pdrag_loss + \
                    paramvdrag*vdrag_loss + parampdfpresmean*pdfpresmean_loss + parampdfvelmean*pdfvelmean_loss

                    allpredictions_aux_pres.append(outputs_AUX_PRES)
                    allpredictions_aux_vel.append(outputs_AUX_VEL)
                    allpredictions.append(outputs)
                    alltargets.append(batchY)
                    allpredictions_pdrag.append(outputs_Pdrag)
                    allpredictions_vdrag.append(outputs_Vdrag)

                    # Backward and optimize
                    optimizer.zero_grad()
                    loss.backward()     
                    optimizer.step()
                    losses.append(loss)
                    epoch_losses_all.append(loss.item())
                    epoch_losses_main.append(loss1.item())
                    epoch_losses_aux1.append(pres_loss.item())
                    epoch_losses_aux2.append(vel_loss.item())
                    epoch_losses_pdrag.append(pdrag_loss.item())
                    epoch_losses_vdrag.append(vdrag_loss.item())

                if epoch%50==0:
                    print("Epoch = {}".format(epoch))

            #Validate For Parameter Tuning
            with torch.no_grad():
                outputsVal,outputsVal_AUX_PRES,outputsVal_AUX_VEL,outputsVal_Pdrag,outputsVal_Vdrag = model(valX)
                lossVal = criterion(outputsVal.squeeze(),valY)
                print("Loss Val = {}".format(lossVal.item()))
                preds=std_scaler_y.inverse_transform(outputsVal.cpu().data.numpy().squeeze())
                tgts=std_scaler_y.inverse_transform(valY.cpu().data.numpy().squeeze())

                aurec = ev_viz.aurec(preds,tgts,y_val[:,-2])
                
                performance={'val_aurec':aurec}

            #Update Settings Dict.
            settings=dict()
            settings['hyperparameters']=hyperparameters
            settings['performance'] = performance
            settings['model'] = model
            
            models.append(settings)
            
            print('\nTraining Complete')


In [ ]:
#Find Best models MSE,MRE,AUREC.
best_model_aurec=None
best_mse=-np.inf
best_mre=-np.inf
best_aurec=-np.inf
for idx,m in enumerate(models):
    hyp=m['hyperparameters']
    perf=m['performance']
    if perf['val_aurec']>best_aurec:
        print("Best AUREC Idx = {}".format(idx))
        best_aurec=perf['val_aurec']
        best_model_aurec=m['model']
    

    print("Hyper Parameters = {}".format(hyp))
    print("Performance = {}".format(perf))
    print("===========================================\n\n")
    

#Test Best Model AUREC
model=best_model_aurec
# Test MSE (un_normalized) 
auxiliary_predictions_pressure=list()
auxiliary_predictions_velocity=list()
with torch.no_grad():
    testpreds,testpreds_aux_pres,testpreds_aux_vel,outputsPDrag,outputsVDrag = model(testX)
    preds = torch.from_numpy(
        std_scaler_y.inverse_transform(testpreds.cpu().detach().squeeze())
    ).float()

    re=std_scaler_x.inverse_transform(testX.to("cpu").data.numpy())[:,-2]
    sf=std_scaler_x.inverse_transform(testX.to("cpu").data.numpy())[:,-1]

    tgts = torch.from_numpy(
        std_scaler_y.inverse_transform(testY.cpu().data.numpy().squeeze())
    ).float()

    auxiliary_predictions_pressure = std_scaler_pres.inverse_transform(testpreds_aux_pres.to("cpu").data.numpy())
    auxiliary_predictions_velocity = std_scaler_vel.inverse_transform(testpreds_aux_vel.to("cpu").data.numpy())

    aurec=ev_viz.aurec(preds.squeeze().data.cpu().numpy(),tgts.squeeze().data.cpu().numpy(),y_test[:,-2])
    print("AUREC = {}".format(aurec))
    print("===============================================================\n")


#SAVE Model and Predictions AUREC
MODEL='PhyDNNAll-PDFPRESMEAN-{}-PDFVELMEAN-{}-PX-TauX'.format(parampdfpresmean,parampdfvelmean)
_tmp=std_scaler_x.inverse_transform(X_test)
preds_pres=std_scaler_pres.inverse_transform(testpreds_aux_pres.cpu().data.numpy())
preds_vel=std_scaler_vel.inverse_transform(testpreds_aux_vel.cpu().data.numpy())
act_pres=std_scaler_pres.inverse_transform(testY_AUX1.cpu().data.numpy())
act_vel=std_scaler_vel.inverse_transform(testY_AUX2.cpu().data.numpy())
preds_pdrag=std_scaler_presdragcomponents.inverse_transform(outputsPDrag.cpu().data.numpy())
preds_vdrag=std_scaler_veldragcomponents.inverse_transform(outputsVDrag.cpu().data.numpy())
act_pdrag=std_scaler_presdragcomponents.inverse_transform(testY_PDrag.cpu().data.numpy())
act_vdrag=std_scaler_veldragcomponents.inverse_transform(testY_VDrag.cpu().data.numpy())


#Structure of output predictions dataframe: 
"""
[  
   Fx-Nondim_Prediction,
   Fx-Nondim_Target,
   Mean_Per_Re_Sf,
   Re,
   Sf,
   pressurepred1,
   ...
   pressurepred10,
   velpred1,
   ...
   velpred10,
   pressureactual1,
   ...
   pressureactual10,
   velactual1,
   ...
   velactual10,
   Px_pred,
   Py_pred,
   Pz_pred
   TauX_pred,
   TauY_pred,
   TauZ_pred,
   Px_Actual,
   Py_Actual,
   Pz_Actual,
   TauX_Actual,
   TauY_Actual,
   TauZ_Actual
]

"""

_tmp2=np.hstack([preds.to("cpu").data.numpy().ravel()[:,None],
                 tgts.to("cpu").data.numpy().ravel()[:,None],
                 y_test[:,-2].squeeze()[:,None],
                 _tmp[:,-2].squeeze()[:,None],
                 _tmp[:,-1].squeeze()[:,None],
                 preds_pres,preds_vel,act_pres,act_vel,preds_pdrag,preds_vdrag,act_pdrag,act_vdrag])

pres_cols=['PressurePredictions_{}'.format(i+1) for i in range(10)]
vel_cols=['VelocityPredictions_{}'.format(i+1) for i in range(10)]
pres_act_cols=['PressureActual_{}'.format(i+1) for i in range(10)]
vel_act_cols=['VelocityActual_{}'.format(i+1) for i in range(10)]
pdrag_pred_cols=['Px_Predictions','Py_Predictions','Pz_Predictions']
vdrag_pred_cols=['Taux_Predictions','Tauy_Predictions','Tauz_Predictions']
pdrag_act_cols=['Px','Py','Pz']
vdrag_act_cols=['Taux','Tauy','Tauz']

_df=pd.DataFrame(_tmp2,columns=['{}-Predictions'.format(MODEL),'Targets','Mean','Re','Solidfraction']+\
                 pres_cols+vel_cols+pres_act_cols+vel_act_cols+pdrag_pred_cols+vdrag_pred_cols+\
                 pdrag_act_cols+vdrag_act_cols)


outputfile="/home/nik90/experiments/particleDragForce/dnn_mt_seq/{}_case1_predictions_TRAINPERCENTAGE_{}.csv".format(MODEL,TRAINPERCENTAGE)
_df.to_csv(outputfile,index=False)
#Save Model
#torch.save(model.state_dict(),"../../../../models/Nikhil_{}_Case1_TrainPercentage_{}".format(MODEL,TRAINPERCENTAGE))